In [2]:
import torch

vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [6]:
with open ("the-verdict.txt" , 'r' , encoding='utf-8') as f:
    raw_text = f.read()

In [7]:
from torch.utils.data import Dataset , DataLoader

class GPTDataset1(Dataset):
    def __init__(self , txt , tokenizer , max_length , stride):
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(txt , allowed_special={"<|endoftext|>"})

        # use a sliding window to chunk 

        for i in range(0 , len(token_ids)- max_length , stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(target_chunk))

    def __len__(self):
            return len(self.input_ids) 

    def __getitem__(self , idx):
            return self.input_ids[idx] , self.output_ids[idx]       

In [12]:
import tiktoken

In [13]:
def create_dataloader1(txt , batch_size=4 , max_length=256 ,stride=128 , shuffle=True , drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    dataset =  GPTDataset1(txt , tokenizer , max_length , stride)

    #create dataset
    dataloader = DataLoader(
        dataset ,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers = num_workers
    )
    return dataloader




In [14]:
max_length =4
dataloader = create_dataloader1(
    raw_text , batch_size=8 , max_length = max_length , stride = max_length , shuffle=False
)

data_iter = iter(dataloader)
inputs , targets = next(data_iter)

In [15]:
print(inputs)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [17]:
token_emb = token_embedding_layer(inputs)

In [18]:
token_emb.shape

torch.Size([8, 4, 256])

In [19]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length , output_dim)

In [22]:
pos_emb = pos_embedding_layer(torch.arange(max_length))
print(pos_emb.shape)

torch.Size([4, 256])


In [24]:
input_emb = token_emb + pos_emb
print(input_emb.shape)

torch.Size([8, 4, 256])
